 Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and limitations under the License.

This section shows you how to upload Vectors into a Postgres table and run simple search queries using the SQL syntax.

In this example, you use a dataset from a CSV file that contains a list of books in different genres. Pgvector will serve as a search engine.

Install a Postgres database connector, a Pgvector module to support vector types with INSERT commands, and a fastembed library to vectorize the data.

Install kubectl and the Google Cloud SDK with the necessary authentication plugin for Google Kubernetes Engine (GKE).

In [ ]:
%%bash

curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl
apt-get update && apt-get install apt-transport-https ca-certificates gnupg
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo gpg --dearmor -o /usr/share/keyrings/cloud.google.gpg
echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] https://packages.cloud.google.com/apt cloud-sdk main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list
apt-get update && sudo apt-get install google-cloud-cli-gke-gcloud-auth-plugin

Install the Postgres client

In [ ]:
! pip install --upgrade --upgrade-strategy only-if-needed pgvector psycopg-binary psycopg fastembed python-dotenv

Replace \<CLUSTER_NAME> with your cluster name, e.g. postgres-cluster. Retrieve the GKE cluster's credentials using the gcloud command.

In [ ]:
%%bash

export KUBERNETES_CLUSTER_NAME=<CLUSTER_NAME>
gcloud container clusters get-credentials $KUBERNETES_CLUSTER_NAME --region $GOOGLE_CLOUD_REGION

  Download the dataset from Git.

In [ ]:
%%bash

export DATASET_PATH=https://raw.githubusercontent.com/epam/kubernetes-engine-samples/internal_lb/databases/postgres-pgvector/manifests/02-notebook/dataset.csv
curl -s -LO $DATASET_PATH

Please run the next command and check if PGvector internal load balancer achieved an IP address. If you see ip address in the output proceed to the next step if blanc please repeat the command after a few minutes or check the status of pg-ilb service from your console, proceed to the next step only when IP address appears.

In [ ]:
%%bash
kubectl get svc pg-ilb -n pg-ns --output jsonpath="{.status.loadBalancer.ingress[0].ip}"

Create an .env file with environment variables required for connecting to PGvector in a Kubernetes cluster.

In [ ]:
%%bash
echo PGVECTOR_ENDPOINT=$(kubectl get svc pg-ilb -n pg-ns --output jsonpath="{.status.loadBalancer.ingress[0].ip}") > .env
echo CLIENTUSERNAME=$(kubectl get secret gke-pg-cluster-superuser -n pg-ns --template='{{index  .data "username"}}'| base64 -d) >> .env
echo CLIENTPASSWORD=$(kubectl get secret gke-pg-cluster-superuser -n pg-ns --template='{{index  .data "password"}}'| base64 -d) >> .env

Import required python libraries:

In [6]:
from dotenv import load_dotenv
from pgvector.psycopg import register_vector
import psycopg
import os
import sys
import csv
from fastembed import TextEmbedding
from typing import List
import numpy as np

Connect to the Postgres cluster:

In [21]:
load_dotenv()
conn = psycopg.connect(
    dbname="app",
    host=os.getenv("PGVECTOR_ENDPOINT"),
    user=os.getenv("CLIENTUSERNAME"),
    password=os.getenv("CLIENTPASSWORD"))

Ensure the pgvector is installed and prepare the table for vectors:

In [ ]:
conn.execute('CREATE EXTENSION IF NOT EXISTS vector;')
register_vector(conn)
conn.execute('DROP TABLE IF EXISTS documents;')
conn.execute('CREATE TABLE documents (id bigserial PRIMARY KEY, author text, title text, description text, embedding vector(384));')

Load data from a CSV: It reads the dataset.csv file for inserting data into a Postgres table.

In [24]:
books = [*csv.DictReader(open('/content/dataset.csv'))]

Declare the Embedding model and vectorize books descriptions from the dataset:

In [25]:
descriptions = [doc["description"] for doc in books]
embedding_model = TextEmbedding(model_name="BAAI/bge-small-en")
embeddings: List[np.ndarray] = list(embedding_model.embed(descriptions))

Insert data (author, title, book description and the corresponding vector) into the Postgres table:

In [ ]:
for i, doc in enumerate(books):
    conn.execute('INSERT INTO documents (author, title, description, embedding) VALUES (%s, %s, %s, %s)', (doc["author"], doc["title"], doc["description"], embeddings[i]))

Query the Postgres database. It runs a search query about `drama about people and unhappy love` and displays results.

This query performs a semantic search against the documents table in Postgres, retrieving a maximum of two results with highest match score relevant to your query text.
It prints each result separated by a line of dashes, in the following format:

- Title: Title of the book, Author: Author of the book
- Book description as stored in your document's description row.

In [ ]:
query_vector = list(embedding_model.embed(["drama about people and unhappy love"]))[0]
response = conn.execute('SELECT title, author, description FROM documents ORDER BY embedding <-> %s LIMIT 2', (query_vector,)).fetchall()
for hit in response:
    print("Title: {}, Author: {}".format(hit[0], hit[1]))
    print(hit[2])
    print("---------")